In [ ]:
from utils.binaries import *
from utils.plotting import *

from utils.Auger.FD import XYComparisonPlot

In [ ]:
import pickle

with open("/cr/users/filip/Projects/XYScanner/all_heat_measurements.pickle", "rb") as f:
    XY_measurements = pickle.load(f)

In [ ]:
save_dir = "/cr/data01/filip/xy-calibration/"

set_plt_style("single")

for telescope, (nov2019, nov2022, oct2023, nov2023) in XY_measurements.items():

    preMC1, preMC2, postMC1 = nov2022, oct2023, nov2023

    fig = XYComparisonPlot(preMC1, preMC2)
    plt.savefig(
        f"/cr/users/filip/plots/XY-heat/xy_comparisons/cal_a_{telescope}_nov2022_oct2023.png"
    )

    # fig = XYComparisonPlot(preMC1, postMC1)
    # plt.savefig(f'/cr/users/filip/plots/XY-heat/xy_comparisons/{telescope}_nov2022_nov2023.png')

    # fig = XYComparisonPlot(preMC2, postMC1)
    # plt.savefig(f'/cr/users/filip/plots/XY-heat/xy_comparisons/{telescope}_oct2023_nov2023.png')

In [ ]:
import utils


def XYRawComparisonPlot(
    *runs: list[dict],
    cmap=plt.cm.coolwarm,
    hist_bins=50,
    vmin=0.6,
    vmax=1.4,
    contrast_boost=False,
) -> plt.figure:
    """Compare the results of two XY runs, normalized to their respective Cal A signal"""

    from matplotlib.colors import Normalize
    from matplotlib.gridspec import GridSpec
    from matplotlib.colorbar import ColorbarBase

    assert len(runs) == 2, "please only compare two runs at a time"

    fig = plt.figure()
    gs = GridSpec(
        2,
        4,
        figure=fig,
        width_ratios=[1, 1, 0.8, 0.05],
        height_ratios=[1, 1],
    )
    gs.update(left=0.05, right=0.95, wspace=0.05, hspace=0.02)

    positions_normalized, pixels_normalized = [], []
    result_dir = "/cr/data01/filip/xy-calibration/results"
    for run in runs:
        xy, CalAs = run["XY"], run["CalA_open_shutter"]
        try:
            CalA_signal = np.zeros(440)
            for CalA in CalAs:
                CalA_signal += np.loadtxt(f"{result_dir}/out_{CalA}.txt", usecols=[2])

        except (IndexError, AssertionError):
            print(
                "Malformed CalA data received, please make sure you pass \
                in two CalAs (pre/post-XY), which have 440 pixels of data"
            )
            return fig

        pixels = np.loadtxt(f"{result_dir}/out_{xy}.txt", usecols=[2])
        positions = pd.read_csv(
            f"{result_dir}/outPositionsComb_{xy}.txt", usecols=["x", "y", "FDeventSum"]
        )

        # pixels /= CalA_signal                                   # normalize pixels to calA pixels
        # positions['FDeventSum'] /= np.sum(CalA_signal)          # normalize positions to calA sum

        positions_normalized.append(positions)
        pixels_normalized.append(pixels)

    pixel_ratio = pixels_normalized[0] / pixels_normalized[1]
    positions_combined = (
        positions_normalized[0]
        .merge(positions_normalized[1], on=["x", "y"])
        .drop_duplicates()
        .dropna()
    )
    positions_ratio = positions_combined.FDeventSum_x / positions_combined.FDeventSum_y
    mean_positions = uncertainties.ufloat(
        np.mean(positions_ratio), np.std(positions_ratio)
    )
    mean_pixels = uncertainties.ufloat(np.mean(pixel_ratio), np.std(pixel_ratio))

    ax1 = fig.add_subplot(gs[:, 0])
    ax2 = fig.add_subplot(gs[:, 1])
    ax3 = fig.add_subplot(gs[0, 2])
    ax4 = fig.add_subplot(gs[1, 2], sharex=ax3)
    ax5 = fig.add_subplot(gs[:, 3])

    # set up colorbar for position and pixel comparison
    norm = Normalize(vmin=vmin, vmax=vmax)
    date1, date2 = runs[0]["date"], runs[1]["date"]
    ColorbarBase(
        ax5,
        cmap=cmap,
        norm=norm,
        orientation="vertical",
        label=rf"$\tilde{{\mathrm{{XY}}}}_\mathrm{{{date1}}}\,/\,\tilde{{\mathrm{{XY}}}}_\mathrm{{{date2}}}$",
    )

    # set up aperture position comparison
    utils.Auger.FD.AperturePlot(ax1)
    c0 = ax1.scatter(
        positions_combined.x,
        positions_combined.y,
        c=positions_ratio,
        norm=norm,
        marker="o",
        cmap=cmap,
        s=4,
    )
    ax1.axis("off")
    ax1.set_title("Aperture view")
    ax1.plot([-650, 650], [1350, 1350], c="k", ls="--", clip_on=False)

    # set up camera pixel comparison
    if contrast_boost:
        _min, _max, mean, std = (
            pixel_ratio.min(),
            pixel_ratio.max(),
            pixel_ratio.mean(),
            pixel_ratio.std(),
        )
        pixel_plot_ratios = np.interp(
            pixel_ratio, (mean - 2 * std, mean + 2 * std), (vmin, vmax)
        )
    else:
        pixel_plot_ratios = pixel_ratio

    utils.Auger.FD.PixelPlot(
        pixel_plot_ratios,
        ax=ax2,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        norm=norm,
        title="Camera view" + (" (contrast boost)" if contrast_boost else ""),
    )
    ax2.plot([-7, 7], [-18, -18], c="r", clip_on=False)

    # histograms for comparison
    bins = np.linspace(vmin, vmax, hist_bins)
    bin_centers = 0.5 * (bins[1:] + bins[:-1])
    ax3.hist(
        np.clip(pixel_ratio, vmin, vmax),
        bins=bins,
        density=True,
        histtype="step",
        color="r",
        ls="solid",
    )
    n1, _, patches = ax3.hist(np.clip(pixel_ratio, vmin, vmax), bins=bins, density=True)
    ax3.text(
        0.02,
        0.98,
        rf"$\langle\tilde{{\mathrm{{XY}}}}^\mathrm{{pix.}}_\mathrm{{{date1}}}\,/\,\tilde{{\mathrm{{XY}}}}^\mathrm{{pix.}}_\mathrm{{{date2}}}\rangle = {mean_pixels.format('S')}$",
        horizontalalignment="left",
        verticalalignment="top",
        transform=ax3.transAxes,
        fontdict={"fontsize": 7},
    )

    for x, p in zip(bin_centers, patches):
        plt.setp(p, "facecolor", cmap(norm(x)))

    ax4.hist(
        np.clip(positions_ratio, vmin, vmax),
        bins=bins,
        density=True,
        histtype="step",
        color="k",
        ls="--",
    )
    n2, _, patches = ax4.hist(
        np.clip(positions_ratio, vmin, vmax), bins=bins, density=True
    )
    ax4.text(
        0.02,
        0.98,
        rf"$\langle\tilde{{\mathrm{{XY}}}}^\mathrm{{pos.}}_\mathrm{{{date1}}}\,/\,\tilde{{\mathrm{{XY}}}}^\mathrm{{pos.}}_\mathrm{{{date2}}}\rangle = {mean_positions.format('S')}$",
        horizontalalignment="left",
        verticalalignment="top",
        transform=ax4.transAxes,
        fontdict={"fontsize": 7},
    )
    for x, p in zip(bin_centers, patches):
        plt.setp(p, "facecolor", cmap(norm(x)))

    ax3.set_ylim(1e-2, 1.2 * max(n1))
    ax4.set_ylim(1e-2, 1.2 * max(n2))
    # ax3.legend(fontsize=7, loc='upper left')
    # ax4.legend(fontsize=7, loc='upper left')
    ax3.set_yticks([])
    ax4.set_yticks([])

    return fig